In [1]:
import gfapy

In [3]:
G = gfapy.Gfa.from_file("../../data/pangenome-graphs/DRB1-3123_unsorted.gfa")

In [7]:
for line in G.segments:
  print(line)

S	1	A
S	2	TTTAACTCCAT
S	3	C
S	4	TTTGAGAAACATTTAATAATGTAATGTGTTTGT
S	5	CATACAGGGTGAA
S	6	TACAGATGCA
S	7	CGGGAGGCCATACGGTTTAGGC
S	8	AAAGGGGAGCACAAAA
S	9	GTTG
S	10	AAGATGAGG
S	11	C
S	12	GCTGCCATCAATGCTGGGACTTCAGGCCAA
S	13	GGGCAG
S	14	GAGCTGAGG
S	15	AAGCCACAA
S	16	GGGAGGACATTTTCTGCAG
S	17	TTGCC
S	18	GAACCAGTAGCAACCAGGTCC
S	19	TGAGAAAGCC
S	20	CTCTCTTGTGGAAGAAT
S	21	AACTGCCAAGCAGGAAAGCT
S	22	TTTCATCCTGCAAAGC
S	23	C
S	24	GGGGCAGAA
S	25	AGTTC
S	26	TTCCTTGAATGTGGTCATCT
S	27	GCAT
S	28	TTCAGCTCAGGAATCCTGCA
S	29	AAAGACAGAGGAG
S	30	AGTGTTGTTTTCA
S	31	ACCTGGCTC
S	32	T
S	33	ACTAACAGTTT
S	34	CTTTTC
S	35	CCCTCTTTCAA
S	36	GGG
S	37	CTCAGATGAGAGCACTGCAGGAAGAAGAAAAA
S	38	GAAGTTCCTGAGTCTCCCCTGAGCCAATAGTCCC
S	39	GCAGAGCACA
S	40	CCTTTTCTAAGTGGAGAGGAGGA
S	41	A
S	42	TTTTGGTGTAAATTGCCTGATCAGAAATTTGGATCCAA
S	43	AGTCTTTCCTATTAT
S	44	TTCTGTCTCATGCCTTATCACCT
S	45	C
S	46	TACCATCATTCT
S	47	GGTGTGTCCTGAGTTTGTTCCTTCCGGTGGGTTTGTGGTCTCGCTGACTTCCTGCAGACCTTCACAGTGAGTGTTACAGCTCTTAAAGGCGGCAGGGATCCAAAGAGTGGGCAGCAGCAAGATTTATTGT